In [ ]:
#################### biorisk database #######################

In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
reg_names = pd.read_csv('databases/biorisk/reg_names', index_col=None)
reg_names = reg_names['0'].tolist()

In [12]:
# filter FASTA files

from Bio import SeqIO
import re

reg_names_covered = []

def sequence_fetcher(file_prefix, reg_names):
    input_file = "databases/biorisk/" + file_prefix + ".fasta"
    output_file = "databases/biorisk/" + file_prefix + "_reg_filt.fasta"
    output_plus_file = "databases/biorisk/" + file_prefix + "_nonreg_filt.fasta"
    fasta = SeqIO.parse(input_file, "fasta")
    records = ()
    records_plus = ()
    for r in fasta:
        regulated = 0
        for tax in reg_names:
            if bool(re.search(tax, r.description)):
#                print("Yes ", tax)
                if regulated == 0:
                    records = records + (r,)
                    reg_names_covered.append(tax)
                    regulated = 1
        if regulated == 0:
            records_plus = records_plus + (r,)
    SeqIO.write(records, output_file, "fasta")
    SeqIO.write(records_plus, output_plus_file, "fasta")


sequence_fetcher("victors_prot", reg_names)
sequence_fetcher("victors_nt", reg_names)

sequence_fetcher("VFDB_setA_pro", reg_names)
sequence_fetcher("VFDB_setA_nt", reg_names)

sequence_fetcher("VFDB_setB_pro", reg_names)
sequence_fetcher("VFDB_setB_nt", reg_names)

sequence_fetcher("blacklist_nt_dedupe", reg_names)


In [17]:
# process IGSC list

reg_seqs_aus = pd.read_excel("/Users/wheelern/Documents/IGSC_seqs/20160919_GI.xlsx", sheet_name=0)
reg_seqs_eu = pd.read_excel("/Users/wheelern/Documents/IGSC_seqs/20160919_GI.xlsx", sheet_name=1)
reg_seqs_cdc = pd.read_excel("/Users/wheelern/Documents/IGSC_seqs/20160919_GI.xlsx", sheet_name=2)
reg_seqs_usccl = pd.read_excel("/Users/wheelern/Documents/IGSC_seqs/20160919_GI.xlsx", sheet_name=3)
reg_seqs_usitar = pd.read_excel("/Users/wheelern/Documents/IGSC_seqs/20160919_GI.xlsx", sheet_name=4)

output_file = "/Users/wheelern/Documents/IGSC_seqs/reg_seqs.fasta"
if os.path.exists(output_file):
    os.remove(output_file)

f = open(output_file, "a")

def print_seqs(df):
    for i in range(0, reg_seqs_aus.shape[0]):
        reg_names_covered.append(reg_seqs_aus.loc[i,"Organism"])
        f.write(">" + str(reg_seqs_aus.loc[i,"GI"]) + "_" + reg_seqs_aus.loc[i,"Description"] + "\n" + reg_seqs_aus.loc[i,"Sequence"] + "\n")

print_seqs(reg_seqs_aus)
print_seqs(reg_seqs_eu)
print_seqs(reg_seqs_cdc)
print_seqs(reg_seqs_usccl)
print_seqs(reg_seqs_usitar)

f.close()

In [31]:
# process SeqScreen Blacklist database

from interface.utils import *

for i in range(0, blast.shape[0]):
    hit = 0
    for tax in reg_names:
        if bool(re.search(tax, blast.loc[i,'subject title'])):
#             print(tax)
            reg_names_covered.append(tax)
            hit = 1
    if hit == 0:
        print(blast.loc[i,'subject title'])

# print(blast_trim.head())
# blast_trim['% identity'].hist(bins=50)

(490, 16)
38
ROV59267.1 hypothetical protein EGO58_11665, partial [Limosilactobacillus reuteri]
ROV59267.1 hypothetical protein EGO58_11665, partial [Limosilactobacillus reuteri]
ROV59267.1 hypothetical protein EGO58_11665, partial [Limosilactobacillus reuteri]
ROV59267.1 hypothetical protein EGO58_11665, partial [Limosilactobacillus reuteri]
ROV59267.1 hypothetical protein EGO58_11665, partial [Limosilactobacillus reuteri]
ROV59267.1 hypothetical protein EGO58_11665, partial [Limosilactobacillus reuteri]
ROV59267.1 hypothetical protein EGO58_11665, partial [Limosilactobacillus reuteri]
ROV59267.1 hypothetical protein EGO58_11665, partial [Limosilactobacillus reuteri]
KAF9917798.1 hypothetical protein BX616_011277 [Lobosporangium transversale]
KAF9917798.1 hypothetical protein BX616_011277 [Lobosporangium transversale]
KAF9917798.1 hypothetical protein BX616_011277 [Lobosporangium transversale]
KAF9917798.1 hypothetical protein BX616_011277 [Lobosporangium transversale]
KAF9917798.1 hy

In [ ]:
# understanding mmseq clustering 

clusts = pd.read_csv("/Users/wheelern/Documents/IGSC_seqs/biorisk_nt_clust.tsv", sep="\t", header=None)
freqs = clusts[0].value_counts()

fig, ax = plt.subplots()
freqs.hist(ax=ax, bins=100, bottom=0.1)
ax.set_yscale('log')
ax.set_xlabel("Number of members in cluster")

print(freqs)

In [28]:
# species coverage in the biorisk database
print(pd.Series(reg_names_covered).value_counts()[0:30])

# species not covered by the biorisk database
print('\n\nSpecies missed')
for i in set(reg_names):
    if i not in reg_names_covered:
        print(i) 

Staphylococcus aureus                                5970
Escherichia coli                                     4065
Vibrio cholerae                                      3228
Legionella pneumophila                               2561
Clostridium perfringens                              2289
Francisella tularensis                               1750
Salmonella enterica subsp. enterica serovar Typhi    1723
Yersinia pestis                                      1497
Escherichia coli O157:H7                             1387
Coxiella burnetii                                    1364
Burkholderia pseudomallei                            1344
African horse sickness virus                         1266
Escherichia coli O157                                1233
Conus californicus                                   1120
Brucella melitensis                                   806
Ricinus communis                                      795
Clostridium botulinum                                 776
Yersinia pseud